In [101]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tp2datostrain/train.csv
/kaggle/input/tp2datostrain/test.csv


In [102]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from category_encoders import CountEncoder
from sklearn.model_selection import train_test_split
import xgboost as xgb
from category_encoders import CatBoostEncoder

In [103]:
posta = pd.read_csv("/kaggle/input/tp2datostrain/train.csv")
test = pd.read_csv("/kaggle/input/tp2datostrain/test.csv")

In [104]:
posta.drop(columns='id',inplace=True)
test_conId = test.copy()
test.drop(columns='id',inplace=True)

In [105]:
es_para_submit = True
agregar_estimacion_preciom2 = True
if es_para_submit: 
    TAMANIO_TRAIN = 240000
    TAMANIO_TEST = 60000
else:
    TAMANIO_TRAIN = 190000
    TAMANIO_TEST = 50000

In [106]:
if not es_para_submit:
    test = posta.tail(TAMANIO_TEST).copy()
    train = posta.head(TAMANIO_TRAIN).copy()
    test.drop(columns = ['precio'], inplace = True)
else:
    train = posta.copy()

In [107]:
dfCheto = pd.concat([train,test], sort=False)
dfCheto['precio'] = SimpleImputer(strategy='median').fit_transform(dfCheto[['precio']])
df = dfCheto.copy()

In [108]:
df.provincia.fillna('provinciaVacia', inplace = True)
aux2 = df[['ciudad', 'provincia']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby(['provincia']).agg(lambda x:x.provincia.value_counts().index[0]).reset_index().set_index('ciudad')
ciuds = aux2.T.to_dict('records').copy()
ciuds = ciuds[0]
df.ciudad.fillna(df.provincia.map(ciuds), inplace = True)
df.ciudad.fillna('ciudadVacia', inplace=True)

In [109]:
df.tipodepropiedad.fillna('tipoVacio', inplace=True)

In [110]:
aux = df.copy()
aux.descripcion.fillna('',inplace=True)
aux['jardin'] = (aux.descripcion.str.contains('jardin') | aux.descripcion.str.contains('jardín')).astype(int)
aux['vigilancia'] = (aux.descripcion.str.contains('vigilancia') | aux.descripcion.str.contains('seguridad')).astype(int) 
aux['buenaUbicacion'] = (aux.descripcion.str.contains('ubicacion') | aux.descripcion.str.contains('ubicación')).astype(int)
aux['lavadero'] = (aux.descripcion.str.contains('lavadero') | aux.descripcion.str.contains('lavado')).astype(int)
aux['jacuzzi'] = (aux.descripcion.str.contains('jacuzzi')).astype(int)
def buscar_barrioCerrado(string):
    return int(('barrio cerrado' in string) or ('urbanizacion cerrada' in string) or ('urbanización cerrada' in string)
    or ('barrio residencial' in string) or ('country' in string) or ('barrio privado' in string) or ('colonia privada' in string))
aux['barrio_cerrado'] = aux.descripcion.map(lambda x:buscar_barrioCerrado(str(x)))

valor_adicionales = {'gimnasio':4,'usosmultiples':2,'piscina':5,'escuelascercanas':3,'centroscomercialescercanos':3,
                     'buenaUbicacion':2,'vigilancia': 5,'jardin':4,'jacuzzi':3,'lavadero':0.5,'barrio_cerrado':10}
            
for columna in aux.columns:
    if columna in valor_adicionales:
        aux[columna] = aux[columna].map(lambda x: x*valor_adicionales[columna])

aux['adicionales'] = aux[valor_adicionales.keys()].sum(axis=1)
df['adicionales'] = aux['adicionales']

In [111]:
df.adicionales.sample(10)

12651     6.0
120474    0.0
154657    4.5
172636    6.0
37762     0.0
81116     5.0
154597    8.0
224818    6.5
2963      6.0
6146      0.0
Name: adicionales, dtype: float64

In [112]:
aux2 = df[['idzona', 'ciudad']].copy()
aux2.dropna(inplace = True)
aux2 = aux2.groupby('ciudad').agg({'idzona':'median'})
ids = aux2.T.to_dict('records').copy()
ids = ids[0]
df.idzona.fillna(df.ciudad.map(ids), inplace = True)
df['idzona'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['idzona']])

In [113]:
aux = df[['lat', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lat':'mean'})
lats = aux.T.to_dict('records').copy()
lats = lats[0]
df.lat.fillna(df.ciudad.map(lats), inplace = True)
df['lat'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lat']])

In [114]:
aux = df[['lng', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'lng':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.lng.fillna(df.ciudad.map(lngs), inplace = True)
df['lng'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['lng']])

In [115]:
aux = df[['tipodepropiedad', 'habitaciones']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'habitaciones':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.habitaciones.fillna(df.tipodepropiedad.map(habts), inplace = True)
df['habitaciones'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['habitaciones']])

In [116]:
aux = df[['tipodepropiedad', 'garages']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'garages':'median'})
habts = aux.T.to_dict('records').copy()
habts = habts[0]
df.garages.fillna(df.tipodepropiedad.map(habts), inplace = True)
df.garages.fillna(0, inplace=True)

In [117]:
df.drop(columns=['direccion'], inplace=True)

In [118]:
aux = df[['antiguedad', 'ciudad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('ciudad').agg({'antiguedad':'mean'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.antiguedad.fillna(df.ciudad.map(lngs), inplace = True)
df['antiguedad'] = SimpleImputer(strategy='most_frequent').fit_transform(df[['antiguedad']])

In [119]:
aux = df[['banos', 'tipodepropiedad']].copy()
aux.dropna(inplace = True)
aux = aux.groupby('tipodepropiedad').agg({'banos':'median'})
lngs = aux.T.to_dict('records').copy()
lngs = lngs[0]
df.banos.fillna(df.tipodepropiedad.map(lngs), inplace = True)
df.banos.fillna(1, inplace=True)

In [120]:
#df.titulo.fillna('', inplace=True)
#df.descripcion.fillna('', inplace=True)
#df['texto'] = df.titulo + ' ' + df.descripcion
#df['texto'] = df.texto.str.len()
#df.drop(columns = ['descripcion', 'titulo'], inplace = True)

In [121]:
aux = df.copy()
aux.metrostotales.fillna(0, inplace=True)
aux.metroscubiertos.fillna(0, inplace =True)
aux['metrostotalesCorregidos'] = aux[['metrostotales', 'metroscubiertos']].max(axis=1)
aux['metroscubiertosCorregidos'] = aux[['metrostotales', 'metroscubiertos']].min(axis=1)
df['metrostotales'] = aux['metrostotalesCorregidos']
df['metroscubiertos'] = aux['metroscubiertosCorregidos']

In [122]:
df['fecha'] = pd.to_datetime(df['fecha'])
df['dia'] = df.fecha.dt.day
df['mes'] = df.fecha.dt.month
df['anio'] = df.fecha.dt.year
df.drop(columns=['fecha'], inplace=True)

In [123]:
df['publicacionesPorCiudad'] = CountEncoder().fit_transform(df['ciudad'])

In [124]:
df['publicacionesPorProvincia'] = CountEncoder().fit_transform(df['provincia'])

In [125]:
df['publicacionesporZonaYAnio'] = df.groupby(['idzona','anio'])['antiguedad'].transform('count')

In [126]:
aux = df[['mes', 'anio']].copy()
aux['cantidad'] =  1
aux['cantidad'] = aux.groupby(['mes', 'anio']).transform('sum')
df['publicacionesEnMes'] = aux['cantidad']

In [127]:
aux = df[['metrostotales', 'tipodepropiedad']].copy()
aux['metrosPromedioPorTipo'] = aux.groupby('tipodepropiedad').transform('mean')
df['metrosPromedioPorTipo'] = aux['metrosPromedioPorTipo']

In [128]:
aux = df[['metrostotales', 'ciudad']].copy()
aux['metrosPromedioPorCiudad'] = aux.groupby('ciudad').transform('mean')
df['metrosPromedioPorCiudad'] = aux['metrosPromedioPorCiudad']

In [129]:
transformador_categorias = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
])

transfomador_numero = Pipeline(steps=[('a', SimpleImputer(strategy = 'median'))])

In [130]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero, ['antiguedad',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng','adicionales',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad']),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [131]:
model = xgb.XGBRegressor(learning_rate= 0.3,n_estimators= 400,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [132]:
#dtrain = xgb.DMatrix(df.drop(columns=['precio','tipodepropiedad', 'provincia', 'ciudad']).copy(), label=df.precio)
#params = {
#    'max_depth':6,
#    'min_child_weight': 1,
#    'eta':.3,
#    'subsample': 1,
#    'colsample_bytree': 1,
#    'objective':'reg:linear',
#}
#resultados_cv = xgb.cv(params,
#    dtrain,
#    seed=42,
#    nfold=5,
#    num_boost_round=2000,
#    metrics={'mae'},
#    early_stopping_rounds=10)

#resultados_cv.shape[0]

In [133]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()

In [134]:
prueba.drop(columns=['precio'], inplace=True)

In [135]:
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [136]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

In [137]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:44:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [138]:
preds = my_pipeline.predict(train_x)

In [139]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(preds, train_y)

284892.0932762779

In [140]:
precio = posta.precio.tail(TAMANIO_TEST).copy()

In [141]:
preds = my_pipeline.predict(prueba)

In [142]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

592569.173468125


USO LOS PRECIOS ESTIMADOS POR 1RA VEZ

In [143]:
aux = df.tail(TAMANIO_TEST).copy()
aux.precio = preds
df = pd.concat([df.head(TAMANIO_TRAIN),aux], sort=False)

In [144]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [145]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [146]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [147]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [148]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [149]:
aux = df[['provincia','ciudad','idzona','anio','mes','precio','metrostotales','tipodepropiedad']].copy()
aux['preciom2'] = aux.precio/aux.metrostotales
aux['trimestre'] = (aux.mes//3)+1-aux.mes//12
aux['semestre'] = (aux.mes//6)+1-aux.mes//12
#aux = aux.groupby(['provincia','anio','tipodepropiedad'])['preciom2'].agg(['min','max','count','mean','median','std'])
def rellenarCon(dataframe,listaDeParametros):
    dataframe['cuenta'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('count')
    dataframe['estimacion'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('mean')
    nombre_col = 'mediana_'+'_'.join(listaDeParametros)
    dataframe[nombre_col] = dataframe.apply(lambda x: x['estimacion'] if x['cuenta'] >= 10 else 0,axis=1)

if agregar_estimacion_preciom2:
    rellenarCon(aux,['anio','idzona'])
    rellenarCon(aux,['anio','trimestre','ciudad'])
    rellenarCon(aux,['anio','semestre','ciudad'])
    rellenarCon(aux,['anio','ciudad'])
    rellenarCon(aux,['ciudad'])
    rellenarCon(aux,['anio','provincia'])

In [150]:
def devolverEstimacion(x,columnas):
    for i in range(len(columnas)):
        if columnas[i].find('mediana') > -1 and x[i]>0: return x[i]
    return 0
if agregar_estimacion_preciom2:
    aux['estimacionM2'] = aux.apply(lambda x: devolverEstimacion(x=x,columnas=aux.columns),axis=1)
    df['estimacion_preciom2'] = aux['estimacionM2']
#df['estimacion_anio_idzona'] = aux['mediana_anio_idzona']
#df['estimacion_ciudad_anio_tipodepropiedad'] = aux['mediana_ciudad_anio_tipodepropiedad']
#df['estimacion_trimestre_idzona'] = aux['mediana_trimestre_idzona']
#df['estimacion_trimestre_ciudad'] = aux['mediana_trimestre_ciudad']

In [151]:
#df.drop(columns=['mes-anio'], inplace = True)

In [152]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [153]:
df.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'ciudad', 'provincia',
       'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'adicionales', 'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona', 'mes-anio',
       'precioPromedioM2PorMes', 'estimacion_preciom2'],
      dtype='object')

In [154]:
model = xgb.XGBRegressor(learning_rate= 0.18,n_estimators= 700,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [155]:
columnas_num = ['antiguedad','estimacion_preciom2',
       'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales',
       'idzona', 'lat', 'lng', 'adicionales',
       'dia', 'mes', 'anio','publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad','catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia','precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona',
       'precioPromedioM2PorMes']
if not agregar_estimacion_preciom2: columnas_num.remove('estimacion_preciom2')
                                     
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero,columnas_num),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [156]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [157]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:46:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [158]:
preds = my_pipeline.predict(train_x)

In [159]:
mean_absolute_error(preds, train_y)

170264.68097114

In [160]:
preds = my_pipeline.predict(prueba)

In [161]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

544331.8706654394


USO LOS PRECIOS ESTIMADOS POR 2DA VEZ

In [162]:
aux = df.tail(TAMANIO_TEST).copy()
aux.precio = preds
df = pd.concat([df.head(TAMANIO_TRAIN),aux], sort=False)

In [163]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [164]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [165]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [166]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [167]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [168]:
aux = df[['provincia','ciudad','idzona','anio','mes','precio','metrostotales','tipodepropiedad']].copy()
aux['preciom2'] = aux.precio/aux.metrostotales
aux['trimestre'] = (aux.mes//3)+1-aux.mes//12
aux['semestre'] = (aux.mes//6)+1-aux.mes//12
#aux = aux.groupby(['provincia','anio','tipodepropiedad'])['preciom2'].agg(['min','max','count','mean','median','std'])
def rellenarCon(dataframe,listaDeParametros):
    dataframe['cuenta'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('count')
    dataframe['estimacion'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('mean')
    nombre_col = 'mediana_'+'_'.join(listaDeParametros)
    dataframe[nombre_col] = dataframe.apply(lambda x: x['estimacion'] if x['cuenta'] >= 10 else 0,axis=1)

if agregar_estimacion_preciom2:
    rellenarCon(aux,['anio','idzona'])
    rellenarCon(aux,['anio','trimestre','ciudad'])
    rellenarCon(aux,['anio','semestre','ciudad'])
    rellenarCon(aux,['anio','ciudad'])
    rellenarCon(aux,['ciudad'])
    rellenarCon(aux,['anio','provincia'])

In [169]:
def devolverEstimacion(x,columnas):
    for i in range(len(columnas)):
        if columnas[i].find('mediana') > -1 and x[i]>0: return x[i]
    return 0

if agregar_estimacion_preciom2:
    aux['estimacionM2'] = aux.apply(lambda x: devolverEstimacion(x=x,columnas=aux.columns),axis=1)
    df['estimacion_preciom2'] = aux['estimacionM2']
#df['estimacion_anio_idzona'] = aux['mediana_anio_idzona']
#df['estimacion_ciudad_anio_tipodepropiedad'] = aux['mediana_ciudad_anio_tipodepropiedad']
#df['estimacion_trimestre_idzona'] = aux['mediana_trimestre_idzona']
#df['estimacion_trimestre_ciudad'] = aux['mediana_trimestre_ciudad']

In [170]:
#df.drop(columns=['mes-anio'], inplace = True)

In [171]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [172]:
df.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'ciudad', 'provincia',
       'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'adicionales', 'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona', 'mes-anio',
       'precioPromedioM2PorMes', 'estimacion_preciom2'],
      dtype='object')

In [173]:
model = xgb.XGBRegressor(learning_rate= 0.1,n_estimators= 1000,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [174]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero,columnas_num),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [175]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [176]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:48:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [177]:
preds = my_pipeline.predict(train_x)

In [178]:
mean_absolute_error(preds, train_y)

193221.51040522204

In [179]:
preds = my_pipeline.predict(prueba)

In [180]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

530126.1721779786


USO LOS PRECIOS ESTIMADOS POR 3RA VEZ

In [181]:
aux = df.tail(TAMANIO_TEST).copy()
aux.precio = preds
df = pd.concat([df.head(TAMANIO_TRAIN),aux], sort=False)

In [182]:
df['catBoostCiudad'] = CatBoostEncoder().fit_transform(df[['ciudad']], df['precio'])
df['catBoostTipo'] = CatBoostEncoder().fit_transform(df[['tipodepropiedad']], df['precio'])
df['catBoostProvincia'] = CatBoostEncoder().fit_transform(df[['provincia']], df['precio'])

In [183]:
aux = df[['precio', 'ciudad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('ciudad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorCiudad'] = df.ciudad.map(precioPromedio)

In [184]:
aux = df[['precio', 'tipodepropiedad', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('tipodepropiedad').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorTipo'] = df.tipodepropiedad.map(precioPromedio)

In [185]:
aux = df[['precio', 'idzona', 'metrostotales']].copy()
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('idzona').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['precioPromedioM2PorZona'] = df.idzona.map(precioPromedio)

In [186]:
aux = df[['precio', 'mes', 'anio', 'metrostotales']].copy()
aux['mes-anio'] = aux.mes + aux.anio
aux['precioMetro'] = aux['precio']/aux['metrostotales']
aux = aux.groupby('mes-anio').agg({'precioMetro':'mean'})
precioPromedio = aux.T.to_dict('records').copy()
precioPromedio = precioPromedio[0]
df['mes-anio'] = df.mes + df.anio
df['precioPromedioM2PorMes'] = df['mes-anio'].map(precioPromedio)

In [187]:
aux = df[['provincia','ciudad','idzona','anio','mes','precio','metrostotales','tipodepropiedad']].copy()
aux['preciom2'] = aux.precio/aux.metrostotales
aux['trimestre'] = (aux.mes//3)+1-aux.mes//12
aux['semestre'] = (aux.mes//6)+1-aux.mes//12
#aux = aux.groupby(['provincia','anio','tipodepropiedad'])['preciom2'].agg(['min','max','count','mean','median','std'])
def rellenarCon(dataframe,listaDeParametros):
    dataframe['cuenta'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('count')
    dataframe['estimacion'] = dataframe.groupby(listaDeParametros)['preciom2'].transform('mean')
    nombre_col = 'mediana_'+'_'.join(listaDeParametros)
    dataframe[nombre_col] = dataframe.apply(lambda x: x['estimacion'] if x['cuenta'] >= 10 else 0,axis=1)

if agregar_estimacion_preciom2:
    rellenarCon(aux,['anio','idzona'])
    rellenarCon(aux,['anio','trimestre','ciudad'])
    rellenarCon(aux,['anio','semestre','ciudad'])
    rellenarCon(aux,['anio','ciudad'])
    rellenarCon(aux,['ciudad'])
    rellenarCon(aux,['anio','provincia'])

In [188]:
def devolverEstimacion(x,columnas):
    for i in range(len(columnas)):
        if columnas[i].find('mediana') > -1 and x[i]>0: return x[i]
    return 0
if agregar_estimacion_preciom2:
    aux['estimacionM2'] = aux.apply(lambda x: devolverEstimacion(x=x,columnas=aux.columns),axis=1)
    df['estimacion_preciom2'] = aux['estimacionM2']
#df['estimacion_anio_idzona'] = aux['mediana_anio_idzona']
#df['estimacion_ciudad_anio_tipodepropiedad'] = aux['mediana_ciudad_anio_tipodepropiedad']
#df['estimacion_trimestre_idzona'] = aux['mediana_trimestre_idzona']
#df['estimacion_trimestre_ciudad'] = aux['mediana_trimestre_ciudad']

In [189]:
#df.drop(columns=['mes-anio'], inplace = True)

In [190]:
entrenamiento = df.head(TAMANIO_TRAIN).copy()
prueba = df.tail(TAMANIO_TEST).copy()
prueba.drop(columns=['precio'], inplace=True)
train_x = entrenamiento.drop(columns=['precio']).copy()
train_y = entrenamiento.precio.copy()

In [191]:
df.columns

Index(['titulo', 'descripcion', 'tipodepropiedad', 'ciudad', 'provincia',
       'antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'idzona', 'lat', 'lng', 'gimnasio', 'usosmultiples',
       'piscina', 'escuelascercanas', 'centroscomercialescercanos', 'precio',
       'adicionales', 'dia', 'mes', 'anio', 'publicacionesPorCiudad',
       'publicacionesPorProvincia', 'publicacionesporZonaYAnio',
       'publicacionesEnMes', 'metrosPromedioPorTipo',
       'metrosPromedioPorCiudad', 'catBoostCiudad', 'catBoostTipo',
       'catBoostProvincia', 'precioPromedioM2PorCiudad',
       'precioPromedioM2PorTipo', 'precioPromedioM2PorZona', 'mes-anio',
       'precioPromedioM2PorMes', 'estimacion_preciom2'],
      dtype='object')

In [192]:
model = xgb.XGBRegressor(learning_rate= 0.03,n_estimators= 1400,scale_pos_weight= 2,max_depth= 10,min_child_weight= 5,gamma= 0.0,colsample_bytree= 0.75,
                     subsample= 0.7,colsample_bylevel= 0.65, tree_method='gpu_hist', gpu_id=0)

In [193]:
preprocessor2 = ColumnTransformer(
    transformers=[
        ('num', transfomador_numero,columnas_num),
        ('cat', transformador_categorias, ['tipodepropiedad', 'provincia', 'ciudad'])
    ])

In [194]:
train_x.tipodepropiedad = train_x.tipodepropiedad.astype(str)
train_x.provincia = train_x.provincia.astype(str)
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor2),
                              ('model', model)
                             ])

In [195]:
my_pipeline.fit(train_x, train_y)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[16:51:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('a',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=None,
                                                                                 missing_values=nan,
                                                                                 strategy='median',
                                                                   

In [196]:
preds = my_pipeline.predict(train_x)

In [197]:
mean_absolute_error(preds, train_y)

303219.96413680096

In [198]:
preds = my_pipeline.predict(prueba)

In [199]:
if not es_para_submit:
    print(mean_absolute_error(preds, precio))

518416.7077682031


In [200]:
# RMSLE=1.0249284784393988 ?
# import the modules we'll need
from IPython.display import HTML
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "knn-results.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
if es_para_submit: 
    res = pd.DataFrame(preds, index=test_conId.id, columns=['precio'])
    res.reset_index(inplace=True)
    res.columns = ['id',"target"]
    display(res.head())
create_download_link(res)

NameError: name 'res' is not defined